Implementation of neural network based on 
Qiang Liu and Dilin Wang, [*Stein Variational Gradient Descent (SVGD): A General Purpose Bayesian Inference Algorithm*](https://arxiv.org/pdf/1608.04471.pdf), NIPS, 2016."

In [2]:
import torch
import random
import math
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
from torch.distributions.gamma import Gamma
from torch.optim import Adam
from sklearn.model_selection import train_test_split

In [3]:
class RBF(torch.nn.Module):
    def __init__(self, h=-1):
        super(RBF, self).__init__()
        self.h = h
    
    def forward(self, theta):
        #theta shape 
        theta_i = theta[:,None,:]  #shape (N,D) -> N,1,D
        theta_j = theta[None, :,:] #shape (N,D) -> 1,N,D
        
        pairwise_dists = torch.norm((theta_i - theta_j),dim=2).pow(2)  #[i-j]^2 i-j.shape = (N,N,D)
        #print(pairwise_dists.shape) N,N
        #pairwise_dists = torch.norm(theta[:,None,:] - theta, dim=2).pow(2)
        h = self.h
        if(h<0):
            #do the meadian trick
            #h = np.median(pairwise_dists)
            p_dists = pairwise_dists.detach().flatten()
            h=torch.median(p_dists)
            #h = torch.median(p_dists)/np.log(theta.shape[0])
            h = torch.sqrt(0.5*h/np.log(theta.shape[0]+1))

        #compute the rbf kernel
        Kxy = torch.exp(-pairwise_dists / h**2 / 2)
        
        return Kxy,h

     
    def updsate(self, model, theta):
        n = theta.shape[0]
        theta = theta.detach().requires_grad_(True)

        logp = model.logp(theta)   
        dlogp = torch.autograd.grad(logp.sum(), theta)[0] 

        Kxy, h = self.forward(theta)
        
        dxKxy = torch.autograd.grad(Kxy.sum(), theta)[0]/(h ** 2)

        dxKxy = -(Kxy.mm(dlogp) + dxKxy) / n #question to the original code

        return dxKxy    
           

In [4]:
class svgd_nn:
    '''
    define a one hidden-layer neural network
    Input:
    batch size, iter, M, n_hiddenm a0, b0, master_stepsize
    In the paper, they take 1 hidden layer with 50 units, 
    the results are averaged over 20 random trials. 
    Use ReLu as activa function
    Use AdaGrad with momentum, 20 particles.
    Mini-batch size is 100
    '''
    def __init__(self, X, y, batch_size, n_hidden, max_iter,  M, a0=1.0, b0=0.1):#, a0, b0, master_stepsize):
        super(svgd_nn, self).__init__()
        
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.n_hidden = n_hidden
        self.M = M
        self.d = X.shape[1]
        self.a0 = a0
        self.b0 = b0
        #self.gamma_prior = Gamma(torch.tensor(1., device=device), torch.tensor(1/0.1, device = device))
        #self.lambda_prior = Gamma(torch.tensor(1., device=device), torch.tensor(1/0.1, device = device))
        
    def forward(self, x, theta):
        ##w1: d*n_hidden; b1: n_hidden; w2 = n_hidden; b2 = 1; 2 variances
        w1 = theta[:, 0:self.d * self.n_hidden].reshape(-1, self.d, self.n_hidden)
        b1 = theta[:, self.d * self.n_hidden:(self.d + 1) * self.n_hidden].unsqueeze(1)
        w2 = theta[:, (self.d + 1) * self.n_hidden:(self.d + 2) * self.n_hidden].unsqueeze(2)
        b2 = theta[:, -3].reshape(-1, 1, 1)
        
        x = x.unsqueeze(0).repeat(self.M, 1,1)  #need to do forward for all particles
        x = F.relu(torch.bmm(x,w1)+b1)
        x = torch.bmm(x,w2)+b2
        y = x.squeeze()
        return y
        
    def logp(self, theta):
        '''
        p(y | W, X, \gamma) = \prod_i^N  N(y_i | f(x_i; W), \gamma^{-1})
        p(W | \lambda) = \prod_i N(w_i | 0, \lambda^{-1})
        p(\gamma) = Gamma(\gamma | a0, b0) 
        p(\lambda) = Gamma(\lambda | a0, b0)
    
        The posterior distribution is as follows:
        p(W, \gamma, \lambda) = p(y | W, X, \gamma) p(W | \lambda) p(\gamma) p(\lambda) 
        '''
        #prepare parameters
        log_gamma = torch.exp(theta[:, -2])  #variance related to gamma
        log_lambda = torch.exp(theta[:, -1])
    
        
        #p(\gamma) = Gamma(\gamma | a0, b0) 
        p_gamma = Gamma(torch.tensor(self.a0, device=device), torch.tensor(1/self.b0, device = device))
        p_lambda = Gamma(torch.tensor(self.a0, device=device), torch.tensor(1/self.b0, device = device))

        
        #get different distributions
        #p(W | \lambda) = \prod_i N(w_i | 0, \lambda^{-1})
        w = theta[:, :-1]
        #p_w = Normal(0, torch.ones_like(log_lambda) / log_lambda)
        p_w = Normal(0, torch.sqrt(torch.ones_like(log_lambda) / log_lambda))

        #p(y | W, X, \gamma) = \prod_i^N  N(y_i | f(x_i; W), \gamma^{-1})
        batch_index = random.sample([i for i in range(self.X.shape[0])], self.batch_size)#mini batch with random idx
        X_batch, y_batch  = self.X[batch_index],self.y[batch_index]               
        outputs = self.forward(X_batch, theta)  #apply the network,[M, batch_size]
        
        pgamma_repeat = log_gamma.unsqueeze(1).repeat(1, self.batch_size)    
        y_batch_repeat = y_batch.unsqueeze(0).repeat(self.M, 1)
        p_y = Normal(outputs, torch.sqrt(torch.ones_like(pgamma_repeat) / pgamma_repeat))
        log_p_y = p_y.log_prob(y_batch_repeat).sum(dim=1)

        log_p0 = p_w.log_prob(w.t()).sum(dim=0) + p_gamma.log_prob(log_gamma) + p_lambda.log_prob(log_lambda)
        log_p = log_p_y * (self.X.shape[0] / self.batch_size) +  log_p0
            # algorithm 1,equation 8
        return log_p
        
    

In [5]:
def test(model, X_test, y_test, theta):
    ##test
    temp = model.forward(X_test, theta)
    y_pred = temp.mean(dim=0) #take mean over particles
    rmse_test = torch.norm(y_pred - y_test)/math.sqrt(y_test.shape[0])
    
    #print('test result')
    #print(y_pred)
    #print(y_test)
    print(rmse_test)

In [6]:
print("start")
if __name__ == '__main__':
    device = torch.device('cpu')
    "load data file"
    data = np.loadtxt("boston_housing")
    #print(data.shape)
    X_input = data[ :, range(data.shape[1] - 1) ] #input
    y_input = data[ :, data.shape[1] - 1 ] #result
    X_input = torch.from_numpy(X_input).float().to(device)
    y_input = torch.from_numpy(y_input).float().to(device)
    
    X_train, X_test, y_train, y_test = train_test_split(X_input, y_input, test_size=0.1, random_state=1)
    X_train_mean, X_train_std = torch.mean(X_train, dim=0), torch.std(X_train, dim=0)
    y_train_mean, y_train_std = torch.mean(y_train, dim=0), torch.std(y_train, dim=0)
    
    X_train = (X_train - X_train_mean) / X_train_std
    X_test = (X_test - X_train_mean) / X_train_std
    y_train = (y_train - y_train_mean) / y_train_std
    y_test = (y_test - y_train_mean) / y_train_std
    
    M, batch_size, n_hidden = 100, 200, 50
    n_iter = 5000
    a0 =1.0
    b0 = 0.1
    gamma = np.random.gamma(a0, b0)
    lambdas = np.random.gamma(a0, b0)

    model = svgd_nn(X_train, y_train, batch_size, n_hidden, n_iter,  M, a0, b0)
    kernel = RBF(h=-1)

    theta = torch.cat(
        [torch.zeros([M, X_train.shape[1] * n_hidden ], device=device).normal_(0, math.sqrt(n_hidden)),
         torch.zeros([M, n_hidden ], device=device),
         torch.zeros([M, n_hidden ], device=device).normal_(0, math.sqrt(X_train.shape[1])),
         torch.zeros([M, 1], device=device),
         torch.log(0.1 * torch.ones([M, 2], device=device))], dim=1)

    '''theta = torch.cat(
        [torch.zeros([M, (X_train.shape[1] + 2) * n_hidden + 1], device=device).normal_(0, math.sqrt(n_hidden)),
         torch.log(gamma* torch.ones([M, 2], device=device)),
         torch.log(lambdas* torch.ones([M, 2], device=device))], dim=1)'''
    
    for epoch in range(n_iter):
        optimizer = Adam([theta], lr=1e-3)
        optimizer.zero_grad()
        theta.grad = kernel.updsate(model, theta)
        optimizer.step()
        if epoch % 100 == 0:
            test(model, X_test, y_test, theta)
    
    test(model, X_test, y_test, theta)


start
tensor(28.8955)


/opt/conda/lib/python3.8/site-packages/torch/autograd/__init__.py:202: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729096996/work/c10/cuda/CUDAFunctions.cpp:100.)
  return Variable._execution_engine.run_backward(


tensor(24.2331)
tensor(21.1119)
tensor(19.6167)
tensor(18.9646)
tensor(18.3925)
tensor(17.4626)
tensor(16.1759)
tensor(14.8767)
tensor(13.7404)
tensor(12.6740)
tensor(11.6590)
tensor(10.8030)
tensor(9.9825)
tensor(9.2272)
tensor(8.5218)
tensor(7.9654)
tensor(7.4496)
tensor(6.9665)
tensor(6.5375)
tensor(6.1412)
tensor(5.8263)
tensor(5.4917)
tensor(5.1988)
tensor(4.9160)
tensor(4.7155)
tensor(4.5223)
tensor(4.3178)
tensor(4.1469)
tensor(3.9744)
tensor(3.8497)
tensor(3.7382)
tensor(3.5879)
tensor(3.4809)
tensor(3.3583)
tensor(3.2852)
tensor(3.2047)
tensor(3.1144)
tensor(3.0485)
tensor(2.9757)
tensor(2.8797)
tensor(2.8255)
tensor(2.7546)
tensor(2.7308)
tensor(2.6822)
tensor(2.6515)
tensor(2.5705)
tensor(2.5381)
tensor(2.5069)
tensor(2.4609)
tensor(2.4151)
